## Imports

In [1]:
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import linear_model    # LogisticRegression
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__)

Pandas   1.1.3
Sklearn  0.23.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [2]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../datasets/titanic/"

## Load data

In [3]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)
df.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '../../datasets/titanic/train.csv'

## Check missings

In [ ]:
df.isnull().sum() / len(df)

In [ ]:
df_test.isna().sum() / len(df_test) # Fare has Missings only in TEST !!!

# Preprocessing
For X data:
- We drop Survived because is the target variable
- We drop Name for simplicity. (We could extract Mr, Mrs, ...)
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [ ]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

# Validation Strategy

Simple split:
- 80% for train
- 20% for validation

In [ ]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    stratify = y,   # ALWAYS RECOMMENDED FOR BETTER VALIDATION
    random_state=4  # Recommended for reproducibility
)

# Prepro

In [ ]:
cat_vars  = ['Sex', 'Embarked']
num_vars  = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)

In [ ]:
#x.isnull().sum()

In [ ]:
#x_test.isnull().sum()

In [ ]:
num_preprocessing = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)), # mean, median
    ('scaler', preprocessing.StandardScaler())
])

cat_preporcessing = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

prepro = compose.ColumnTransformer(transformers=[
    ('num', num_preprocessing, num_vars),
    ('cat', cat_preporcessing, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

prepro

In [ ]:
x_train_prepro  = prepro.fit_transform(x_train)  # ONLY FIT THE PREPROCESSOR ON TRAIN TO AVOID DATA LEAKS !!!
x_val_prepro    = prepro.transform(x_val)

x_train_prepro.shape, x_val_prepro.shape

# Check our preprocesed data

In [ ]:
cat_vars_oh = prepro.named_transformers_["cat"]["onehot"].get_feature_names(cat_vars).tolist()
cat_vars_oh

In [ ]:
pd.DataFrame(data = x_train_prepro, columns=num_vars+cat_vars_oh).head() # X DATA (WILL BE TRAIN+VALID DATA)

In [ ]:
pd.DataFrame(data = x_val_prepro, columns=num_vars+cat_vars_oh).head() # X_TEST DATA (NEW DATA)

# Train Model

In [ ]:
model = linear_model.LogisticRegression(max_iter=2000, random_state=1)
model.fit(x_train_prepro, y_train)

## Full Pipeline: Prepro + Model

In [ ]:
full_model = pipeline.Pipeline([('preprocessor', prepro), ('model', model)])
full_model

# Validate

See performance metrics like:
- Accuracy
- Balanced Accuracy
- Confusion matrix

In [ ]:
# Get Predictions
y_pred       = model.predict(x_val_prepro)
y_pred_proba = model.predict_proba(x_val_prepro)[:, 1]

y_pred[:3], y_pred_proba[:3]

In [ ]:
# See metrics
print("Accuracy:          ", metrics.accuracy_score(y_val, y_pred)*100)
print("Balanced accuracy: ", metrics.balanced_accuracy_score(y_val, y_pred)*100)
print("Log loss:          ", metrics.log_loss(y_val, y_pred_proba))
print("AUC:               ", metrics.roc_auc_score(y_val, y_pred_proba)*100) # Area Under ROC Curve


_, axes = plt.subplots(1, 2, figsize=(8, 4))
metrics.plot_confusion_matrix(model, x_val_prepro, y_val,
                      cmap=plt.cm.Blues, normalize=None,
                      ax=axes[0]);

metrics.plot_confusion_matrix(model, x_val_prepro, y_val,
                      cmap=plt.cm.Blues, normalize="true",
                      ax=axes[1]);

## See cofficients (Only for linear models)

In [ ]:
coefs = pd.DataFrame(data=model.coef_[0].tolist(), index=num_vars+cat_vars_oh, columns=["Coefs"]).sort_values(by="Coefs")
coefs

In [ ]:
coefs.plot.barh();